In [13]:
import json
import datetime as dt

import pandas as pd
import pystac
import xarray as xr

In [2]:
catalog = pystac.Catalog(id='N2560_RAL3', description='cat1')

In [4]:
print(json.dumps(catalog.to_dict(), indent=4))

{
    "type": "Catalog",
    "id": "N2560_RAL3",
    "stac_version": "1.1.0",
    "description": "cat1",
    "links": []
}


In [5]:
file = 'http://hackathon-o.s3.jc.rl.ac.uk/sim-data/DYAMOND3_example_data/healpix/toa_outgoing_longwave_flux/hpz10/glm.n2560_RAL3p3.apvera_20200120T00.toa_outgoing_longwave_flux.hpz10.zarr'
ds = xr.open_dataset(file,engine='zarr')

/home/users/mmuetz/miniforge3/envs/hackathon_env/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)
/tmp/ipykernel_727/96578022.py:2: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds = xr.open_dataset(file,engine='zarr')


In [6]:
ds

<xarray.Dataset> Size: 1GB
Dimensions:                     (cell: 12582912, time: 12)
Coordinates:
  * cell                        (cell) int64 101MB 0 1 2 ... 12582910 12582911
    forecast_period             (time) timedelta64[ns] 96B ...
    forecast_reference_time     datetime64[ns] 8B ...
  * time                        (time) datetime64[ns] 96B 2020-01-20T00:02:00...
Data variables:
    toa_outgoing_longwave_flux  (time, cell) float64 1GB ...
Attributes:
    grid_mapping:     healpix_nested
    healpix_zoom:     10
    source:           Data from Met Office Unified Model
    standard_name:    toa_outgoing_longwave_flux
    um_stash_source:  m01s02i205
    um_version:       13.5
    units:            W m-2

In [17]:
item = pystac.Item(
    id='toa_outgoing_longwave_flux',
    start_datetime=pd.Timestamp(ds.time.values[0]).to_pydatetime(),
    end_datetime=pd.Timestamp(ds.time.values[-1]).to_pydatetime()
)

TypeError: Item.__init__() missing 4 required positional arguments: 'geometry', 'bbox', 'datetime', and 'properties'

In [18]:
from shapely.geometry import mapping, box

In [20]:
bbox = [0, -90, 360, 90]  # Global bounding box

In [21]:
geometry = mapping(box(*bbox))  # Convert to GeoJSON format

In [22]:
geometry

{'type': 'Polygon',
 'coordinates': (((360.0, -90.0),
   (360.0, 90.0),
   (0.0, 90.0),
   (0.0, -90.0),
   (360.0, -90.0)),)}

In [26]:
item = pystac.Item(
    id='toa_outgoing_longwave_flux',
    geometry=geometry,
    bbox=bbox,
    datetime=None,
    properties={},
    start_datetime=pd.Timestamp(ds.time.values[0]).to_pydatetime(),
    end_datetime=pd.Timestamp(ds.time.values[-1]).to_pydatetime()
)

In [27]:
item

<Item id=toa_outgoing_longwave_flux>

In [28]:
catalog.add_item(item)

<Link rel=item target=<Item id=toa_outgoing_longwave_flux>>

In [29]:
catalog.describe()

* <Catalog id=N2560_RAL3>
  * <Item id=toa_outgoing_longwave_flux>


In [32]:
item.add_asset(
    key='data',
    asset = pystac.Asset(href='http://hackathon-o.s3.jc.rl.ac.uk/sim-data/DYAMOND3_example_data/healpix/toa_outgoing_longwave_flux/hpz10/glm.n2560_RAL3p3.apvera_20200120T00.toa_outgoing_longwave_flux.hpz10.zarr'),
)

In [33]:
print(json.dumps(catalog.to_dict(), indent=4))

{
    "type": "Catalog",
    "id": "N2560_RAL3",
    "stac_version": "1.1.0",
    "description": "cat1",
    "links": [
        {
            "rel": "item",
            "href": null,
            "type": "application/geo+json"
        }
    ]
}


In [35]:
catalog.normalize_hrefs('')

In [36]:
print(json.dumps(catalog.to_dict(), indent=4))

{
    "type": "Catalog",
    "id": "N2560_RAL3",
    "stac_version": "1.1.0",
    "description": "cat1",
    "links": [
        {
            "rel": "root",
            "href": "/home/users/mmuetz/projects/wcrp_hackathon/notebooks/catalog.json",
            "type": "application/json"
        },
        {
            "rel": "item",
            "href": "/home/users/mmuetz/projects/wcrp_hackathon/notebooks/toa_outgoing_longwave_flux/toa_outgoing_longwave_flux.json",
            "type": "application/geo+json"
        },
        {
            "rel": "self",
            "href": "/home/users/mmuetz/projects/wcrp_hackathon/notebooks/catalog.json",
            "type": "application/json"
        }
    ]
}
